In [1]:
import math
from collections import defaultdict
from datetime import datetime
from sqlalchemy import sql
from database_utils import Session; session = Session();
from models.models import Appointment, Education, Judge, Court, YearParty, Congress, UnsuccessfulNomination
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [21]:
query = (
    session.query(
        UnsuccessfulNomination.president,
        Appointment.party_of_appointing_president,
        UnsuccessfulNomination.congress_start_year,
        sql.func.count(1).label('count')
    )
    .join(Appointment, Appointment.appointing_president == UnsuccessfulNomination.president)
    .group_by(
        UnsuccessfulNomination.president,
        Appointment.party_of_appointing_president,
        UnsuccessfulNomination.congress_start_year,
    )
    .order_by(
        UnsuccessfulNomination.congress_start_year,
        # Because some president's appoint in their lame duck period, the congress is the _next_ congress
        # Use minimum nomination date to sort it so all of president's rows are blocked together
        sql.func.min(UnsuccessfulNomination.nomination_date),
        Appointment.party_of_appointing_president,
        UnsuccessfulNomination.president,
    )
)

In [22]:
session.rollback()
query.all()

[('George Washington', None, 1789, 195),
 ('George Washington', None, 1791, 39),
 ('George Washington', None, 1793, 39),
 ('George Washington', None, 1795, 78),
 ('John Adams', 'Federalist', 1799, 253),
 ('Thomas Jefferson', 'Jeffersonian Republican', 1801, 19),
 ('Thomas Jefferson', 'Jeffersonian Republican', 1803, 19),
 ('Thomas Jefferson', 'Jeffersonian Republican', 1805, 19),
 ('James Madison', 'Jeffersonian Republican', 1809, 42),
 ('James Madison', 'Jeffersonian Republican', 1811, 14),
 ('James Madison', 'Jeffersonian Republican', 1813, 14),
 ('James Madison', 'Jeffersonian Republican', 1815, 14),
 ('James Monroe', 'Jeffersonian Republican', 1821, 23),
 ('John Quincy Adams', 'Jeffersonian Republican', 1825, 26),
 ('John Quincy Adams', 'Jeffersonian Republican', 1827, 39),
 ('Andrew Jackson', 'Democratic', 1833, 52),
 ('Andrew Jackson', 'Democratic', 1835, 26),
 ('John Tyler', 'Whig', 1841, 40),
 ('John Tyler', 'Whig', 1843, 80),
 ('James K. Polk', 'Democratic', 1845, 10),
 ('Zach